# 세션 4 과제

나이키 신발들의 이름 데이터를 받아와서, 리셀 가격을 맞춰보는 NLP 과제입니다. 

In [1]:
# 필요 모듈 임포트

import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings('ignore')

In [2]:
# drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 구글 드라이브에서 데이터셋을 가져오자 
# index_col은 가져올 때 새로운 column이 생기지 않게 하기 위함이다.

df = pd.read_csv('/content/drive/MyDrive/NLP_data', index_col=0)

In [4]:
# 가져온 데이터프레임을 살펴보자

df.head(3)

,name,price_resell
0,Nike Dunk Low Retro Black,152800
1,Nike Air Force 1 '07 Low White,136200
2,Nike Air Force 1 '07 WB Flax,173200


In [5]:
target_data = df['price_resell']

## data_preprocess

In [6]:
# tokenizer에 fit 하기 위해서 데이터 프레임의 name 컬럼의 값을 가져옴.
names = df['name'].values

print(names)

['Nike Dunk Low Retro Black' "Nike Air Force 1 '07 Low White"
 "Nike Air Force 1 '07 WB Flax" ...
 'Nike x Diamond Supply Co. SB Dunk High Tiffany'
 'Nike Roshe One Black Anthracite Sail'
 'Nike Air Force 1 High Premium Le Work Boot']


In [7]:
# sentences로 tokenizer를 fit 해준다.  

tokenizer = Tokenizer()
tokenizer.fit_on_texts(names)
word_index = tokenizer.word_index

print(f"word_index의 크기: {len(word_index)}")

word_index의 크기: 1172


In [8]:
# 문장을 토큰화 한다.
tokenized_name = tokenizer.texts_to_sequences(names)
tokenized_name[:5]

[[1, 7, 5, 17, 9],
 [1, 2, 6, 4, 11, 5, 8],
 [1, 2, 6, 4, 11, 362, 187],
 [1, 3, 40, 2, 6, 4, 5, 187],
 [1, 18, 164, 61, 26, 25, 142, 252, 13, 660, 661]]

In [9]:
# 토큰화 한 문장을 padding 해준다. 
padded_name = pad_sequences(tokenized_name)
padded_name[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   7,   5,
         17,   9],
       [  0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   6,   4,  11,
          5,   8],
       [  0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   6,   4,  11,
        362, 187],
       [  0,   0,   0,   0,   0,   0,   0,   1,   3,  40,   2,   6,   4,
          5, 187],
       [  0,   0,   0,   0,   1,  18, 164,  61,  26,  25, 142, 252,  13,
        660, 661]], dtype=int32)

In [10]:
padded_name.shape

(1455, 15)

## modeling

In [11]:
tf.random.set_seed(42)


# 임베딩 층에 입력될 단어의 수를 지정한다.
word_size = len(word_index) + 1

# train_data, test_data를 나눈다.

train_X, test_X, train_y, test_y = train_test_split(padded_name, target_data, test_size=0.3, random_state=42)

model = keras.models.Sequential([
    # 임베딩층
    keras.layers.Embedding(word_size, word_size, input_length=15),
    # 회귀를 위해 임베딩층의 output값을 1차원의 array로 바꾸어준다. 
    keras.layers.Flatten(),
    keras.layers.Dense(16),
    keras.layers.Dropout(0.5), # 과적합 방지를 위해서 넣는 층.
    keras.layers.Dense(4),
    keras.layers.Dropout(0.5), # 과적합 방지를 위해서 넣는 층.
    keras.layers.Dense(1)
])

# MAPE 를 사용한다.
# 예측 값과의 오차를 % 로 알려주는 손실함수이다.
# 더 직관적으로 오차를 알 수 있다.
model.compile(optimizer='adam', loss= keras.losses.MeanAbsolutePercentageError())
model.fit(train_X, train_y, epochs=100)

Epoch 1/100
32/32 [==============================] - 3s 40ms/step - loss: 99.9807
Epoch 2/100
32/32 [==============================] - 1s 34ms/step - loss: 99.8266
Epoch 3/100
32/32 [==============================] - 1s 27ms/step - loss: 99.2280
Epoch 4/100
32/32 [==============================] - 1s 27ms/step - loss: 97.4999
Epoch 5/100
32/32 [==============================] - 1s 26ms/step - loss: 94.1625
Epoch 6/100
32/32 [==============================] - 1s 26ms/step - loss: 87.5635
Epoch 7/100
32/32 [==============================] - 1s 26ms/step - loss: 75.6198
Epoch 8/100
32/32 [==============================] - 1s 26ms/step - loss: 64.0247
Epoch 9/100
32/32 [==============================] - 1s 25ms/step - loss: 58.3810
Epoch 10/100
32/32 [==============================] - 1s 28ms/step - loss: 56.1280
Epoch 11/100
32/32 [==============================] - 1s 26ms/step - loss: 56.7858
Epoch 12/100
32/32 [==============================] - 1s 27ms/step - loss: 55.7897
Epoch 13/100


## model evaluate

In [12]:
# 모델의 오차는 대략 30% 로 나온다. 
model.evaluate(test_X, test_y)

14/14 [==============================] - 0s 3ms/step - loss: 32.9089


32.908931732177734

## 신발 가격 예측 해보기

과제 하느라 수고하셨습니다! 

아래는 위 학습된 모델을 통해서 예측을 진행해주는 함수입니다.

여러분들이 상상한 신발의 리셀가격은 얼마인지 확인해보세요!

PS) word_index 안에 있는 단어만으로 구성되어야 예측이 더 자연스러워집니다. 

In [57]:
def make_prediction(name):
    tokenized_name = tokenizer.texts_to_sequences([name])
    padded_name = pad_sequences(tokenized_name, 15).reshape((-1,15))
    pred = model.predict(padded_name)
    return pred

In [68]:
make_prediction('Nike x off white x chicago x kobe x stussy')

1/1 [==============================] - 0s 70ms/step


array([[280049.38]], dtype=float32)

In [62]:
word_index.keys()

dict_keys(['nike', 'air', 'x', '1', 'low', 'force', 'dunk', 'white', 'black', 'max', "'07", 'sb', 'and', 'high', 'grey', 'sp', 'retro', 'zoom', 'red', 'blue', 'mid', '2', 'pro', '97', 'light', 'prm', 'sail', 'se', 'green', 'lv8', 'blazer', '95', 'qs', 'orange', 'acg', 'university', 'off', 'lebron', 'next', 'supreme', 'sacai', 'more', 'uptempo', 'flyknit', 'og', 'dark', 'gold', 'of', 'navy', 'trainer', 'ep', '90', 'react', 'plus', 'waffle', 'stussy', 'pink', 'brown', 'triple', 'daybreak', '5', 'gore', 'tex', 'lx', 'racer', 'royal', 'silver', 'purple', '2020', 'tailwind', 'midnight', 'club', 'smoke', 'undercover', 'the', 'pack', 'fear', 'god', 'vapormax', 'volt', 'olive', 'bone', 'mountain', 'fly', '96', '79', 'comme', 'des', 'garcons', 'premium', 'metallic', 'color', 'team', 'summit', '3', 'yellow', 'platinum', "'77", 'element', 'day', 'pure', 'hyper', 'kobe', 'multi', 'varsity', '2018', 'crimson', '98', 'pegasus', '87', 'cortez', 'fog', 'ldwaffle', 'emb', 'zoomx', '2021', 'wolf', '0', 